<a href="https://colab.research.google.com/github/Devesh-Patodia/NLP-J038/blob/class-assignments/LGBM_on_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re


#from sklearn.model_selection import train_test_split as tts
#from sklearn.ensemble import RandomForestClassifier

#from sklearn.preprocessing import LabelEncoder, LabelBinarizer
#from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [2]:
twitter = pd.read_csv('https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv', error_bad_lines=False)
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [3]:
amazon = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz', lines=True)
amazon.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [4]:
twitter['Sentiment'].unique()

array(['positive', 'negative', 'neutral', 'irrelevant'], dtype=object)

In [0]:
def stop(text):
    text=re.sub('[^a-zA-Z]', " ", str(text))  
    return text

In [10]:
twitter['clean_text'] = twitter['TweetText'].apply(stop)
twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,clean_text
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,Now all Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,Hilarious youtube video guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
corpus = twitter['clean_text'] 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names()) 
print(X.toarray()) 

['aa', 'aaa', 'aaargh', 'aac', 'aaddaqa', 'aai', 'aalkhubaizi', 'aan', 'aanbieding', 'aanbod', 'aangekondigd', 'aantal', 'aantikken', 'aap', 'aapl', 'aaron', 'ab', 'abaikan', 'abandonado', 'abandoned', 'abarcar', 'abbas', 'abbatter', 'abbia', 'abbreviations', 'abc', 'abdullah', 'abend', 'aberto', 'abh', 'abhurvyc', 'abia', 'abidin', 'ability', 'able', 'abonnez', 'about', 'aboutthatlife', 'above', 'abra', 'abrass', 'abrazan', 'abrazo', 'abren', 'abri', 'abrir', 'abro', 'abruma', 'absolutely', 'absoluto', 'absorbiendo', 'abst', 'abstract', 'absurdos', 'abt', 'aburre', 'aburrido', 'aburriiiidooo', 'aburrio', 'abusou', 'abxl', 'abzuh', 'ac', 'acaba', 'acaban', 'acabar', 'acabas', 'acabo', 'acabou', 'academy', 'acap', 'acappellamedia', 'acaso', 'acc', 'accelerate', 'accelerated', 'acceleration', 'accent', 'accenture', 'acceptable', 'accesibilidad', 'access', 'accessibilit', 'accessibility', 'accessing', 'accessnetworks', 'accessories', 'accidentalmente', 'accidently', 'accompanying', 'accor

In [0]:
df = pd.DataFrame(X.toarray(),columns=(vectorizer.get_feature_names()))

In [0]:
y = twitter['Sentiment']

In [0]:
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [0]:
x_train, x_test, y_train, y_test = tts(X, y, test_size=0.30, random_state=42)

In [0]:
RFC = RandomForestClassifier()
RFC.fit(x_train, y_train)
y_pred = RFC.predict(x_test)

In [22]:
print(accuracy_score(y_pred, y_test))

0.7522816166883963


In [24]:
y_pred[0]

'neutral'

In [0]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [0]:
def score(y_test, y_pred, average='macro'):
  LB = LabelBinarizer()
  LB.fit(y_test)
  y_test = LB.transform(y_test)
  y_pred = LB.transform(y_pred)
  return roc_auc_score(y_test, y_pred, average=average)

In [30]:
score(y_pred, y_test)

0.8699866192768626

In [34]:
LE = LabelEncoder()
y_test = LE.fit_transform(y_test)

pred_prob = RFC.predict_proba(x_test)
pred_prob

array([[0.21489538, 0.01      , 0.77510462, 0.        ],
       [0.08      , 0.09      , 0.58      , 0.25      ],
       [0.02      , 0.        , 0.98      , 0.        ],
       ...,
       [0.29      , 0.03      , 0.64      , 0.04      ],
       [0.05      , 0.14      , 0.64      , 0.17      ],
       [0.09      , 0.01      , 0.82      , 0.08      ]])

In [0]:
def threshold_6(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [0]:
from sklearn.metrics import  classification_report

In [39]:
print(classification_report((threshold_6(pred_prob)),y_test))
print('\n\n')
score(threshold_6(pred_prob),y_test)

              precision    recall  f1-score   support

           0       0.78      0.92      0.84       415
           1       0.10      0.95      0.18        19
           2       0.96      0.63      0.76      1087
           3       0.09      1.00      0.17        13

    accuracy                           0.72      1534
   macro avg       0.48      0.87      0.49      1534
weighted avg       0.89      0.72      0.77      1534






0.8922181679827811

In [0]:
def threshold_7(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [42]:
print(classification_report((threshold_7(pred_prob)),y_test))
print('\n\n')
score(threshold_7(pred_prob),y_test)

              precision    recall  f1-score   support

           0       0.69      0.94      0.80       360
           1       0.04      0.88      0.07         8
           2       0.97      0.60      0.74      1159
           3       0.05      1.00      0.09         7

    accuracy                           0.69      1534
   macro avg       0.44      0.86      0.43      1534
weighted avg       0.90      0.69      0.75      1534






0.8798366757818918

In [0]:
def threshold_8(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a

In [45]:
print(classification_report((threshold_8(pred_prob)),y_test))
print('\n\n')
score(threshold_8(pred_prob),y_test)

              precision    recall  f1-score   support

           0       0.58      0.98      0.73       292
           1       0.01      1.00      0.01         1
           2       0.99      0.57      0.73      1238
           3       0.02      1.00      0.04         3

    accuracy                           0.65      1534
   macro avg       0.40      0.89      0.38      1534
weighted avg       0.91      0.65      0.72      1534






0.8933862214782706